# Custom attibution models
## Preparing
### Installing programs

We will need the following tools:
* ClickHouse DBMS
* Python 2.7 + libs (pandas, requests, plot.ly)
* Jupyter Notebook

#### ClickHouse Installation
ClickHouse can be installed on Linux (you can find the installation guide here: https://clickhouse.yandex/).

Windows and Mac users can run ClickHouse on their machines using Docker: the first line sets up clickhouse-server on port 8123, the second line allows to connect to the console client.
```
docker run -d --name clickhouse-server --publish=8123:8123 --publish=9000:9000 yandex/clickhouse-server
docker run -it --rm --link clickhouse-server:9000 yandex/clickhouse-client --host clickhouse-server
```

#### Python and Jupyter Notebook Installation

Python can be downloaded from the [official site](https://www.python.org/getit/). 
We will use some other libraries in addition to general ones. The easiest way to install them is to use the package manager pip ([how to install pip](https://pip.pypa.io/en/stable/installing/))

```
pip install pandas jupyter requests plotly
```

To run the Jupyter Notebook, execute the following command in the terminal:

```jupyter notebook```

After that the web interface will be available at http://localhost:8888.

### Loading data to ClickHouse using Logs API

The easiest way to load your site's data into ClickHouse is to use the [script on GitHub](https://github.com/yndx-metrika/logs_api_integration).

First of all, you need to specify some params in the config file.

Config includes basic params, and you will need to set it up just once. For this example I've used the following config file:


```
{
	"token" : "<insert>",
	"counter_id": "6375151",
	"visits_fields": [
	    "ym:s:counterID",
	    "ym:s:dateTime",
	    "ym:s:date",
	    "ym:s:visitDuration",
	    "ym:s:bounce",
	    "ym:s:pageViews",
	    "ym:s:goalsID",
	    "ym:s:clientID",
	    "ym:s:lastDirectConditionType",
	    "ym:s:purchaseRevenue",
	    "ym:s:purchaseID",
	    "ym:s:lastTrafficSource",
	    "ym:s:lastAdvEngine",
	    "ym:s:lastSearchEngineRoot",
	    "ym:s:visitID",
	    "ym:s:UTMMedium",
	    "ym:s:startURL"
	],
	"log_level": "DEBUG",
	"retries": 1,
	"retries_delay": 60,
	"clickhouse": {
		"host": "http://localhost:8123",
		"user": "",
		"password": "",
		"visits_table": "visits_all",
		"hits_table": "hits_all",
		"database": "default"
	}
}
```

If you don't have your own counter yet, you can try Logs API using demo data for counter installed on Yandex.Metrica help page (`CounterID = 29761725`).

Some parameters, such as source (hits or visits) and date range, are specified in CLI.

There are several possible modes for this script.
First of all, you can specify params `start_date` and `end_date` to get data for a particular period:

```python metrica_logs_api.py -source visits -start_date 2015-12-25 -end_date 2015-12-25```

Another option is to use one of the following predefined modes:
* __history__ - loads all data from the beginning of time till the day before yesterday
* __regular__ -  recommended for automated download: this script will load data for the day before yesterday only
* __regular_early__ - the same as __regular__, but it loads data for yesterday (it's recommended to use with regular mode, because visits for yesterday can lack some hits)

```python metrica_logs_api.py -source visits -mode regular```

### Useful functions

Let's write some functions for integration with ClickHouse. The first function just returns a database response. The second function is actually a wrapper for the first one, that returns data as pandas dataframe.

In [1]:
HOST = 'http://localhost:8123'
import requests
import pandas as pd
import StringIO

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

## Warm-up, or a few simple SQL-queries
Let's look at the data we have in the `visits_all` table loaded via the Logs API.

In [2]:
q = '''DESCRIBE visits_all'''
print get_clickhouse_data(q)

CounterID	UInt32		
DateTime	DateTime		
Date	Date		
Duration	UInt32		
IsBounce	UInt8		
PageViews	Int32		
GoalsID	Array(UInt32)		
ClientID	UInt64		
DirectConditionType	UInt8		
PurchaseRevenue	Array(Float64)		
PurchaseID	Array(String)		
LastSearchEngine	String		
LastAdvEngine	String		
LastTraficSource	String		
VisitID	UInt64		
UTMMedium	String		
StartURL	String		
ImpressionsProductID	Array(String)		



In [30]:
q = '''
    SELECT * FROM visits_all 
    LIMIT 5
    FORMAT TabSeparatedWithNames
    '''
get_clickhouse_df(q)

,CounterID,DateTime,Date,Duration,IsBounce,PageViews,GoalsID,ClientID,DirectConditionType,PurchaseRevenue,PurchaseID,LastSearchEngine,LastAdvEngine,LastTraficSource,VisitID,UTMMedium,StartURL,ImpressionsProductID
0,6375151,2015-12-01 09:31:49,2015-12-01,4,1,1,[],0,0,[],[],yandex,market,ad,5934905382539739018,cpc,http://220-volt.ru/catalog-156860/?ref=yamar&y...,['156860']
1,6375151,2015-12-01 21:33:12,2015-12-01,0,1,1,[],0,0,[],[],yandex,NaN,organic,5935082670770524045,NaN,http://m.220-volt.ru/catalog/svetodiodnyj-zana...,[]
2,6375151,2015-12-01 08:54:22,2015-12-01,1,1,1,[],0,0,[],[],yandex,market,ad,5934896179407679609,cpc,http://220-volt.ru/catalog-210053/?ref=yamar&y...,['210053']
3,6375151,2015-12-01 10:56:43,2015-12-01,0,1,1,[],0,0,[],[],NaN,NaN,direct,5934926249142175607,NaN,http://ulyanovsk.220-volt.ru/catalog-59739/,[]
4,6375151,2015-12-01 15:25:47,2015-12-01,171,0,6,[],0,0,[],[],NaN,NaN,direct,5934992373385930594,NaN,http://m.220-volt.ru/catalog/2-55/,[]


In [4]:
q = '''
    SELECT min(Date), max(Date)
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,min(Date),max(Date)
0,2015-12-01,2015-12-31


Let's calculate the total number of visits, users, purchases, and revenue.

__Disclaimer: revenue was multiplied by a random number.__

In [5]:
q = '''
    SELECT 
        count() as TotalVisits,
        uniq(ClientID) as TotalUsers,
        sum(length(PurchaseID)) as TotalPurchases,
        sum(arraySum(PurchaseRevenue)) as TotalRevenue
    FROM visits_all
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,TotalVisits,TotalUsers,TotalPurchases,TotalRevenue
0,5451184,2976586,70464,8336006970


## Paths to conversion (purchase)

First of all, let's look at visits with purchases.

In [6]:
q = '''
    SELECT 
        LastTraficSource,
        ClientID,
        VisitID
    FROM visits_all
    WHERE (ClientID != 0) AND (length(PurchaseID) != 0)
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,LastTraficSource,ClientID,VisitID
0,direct,1450095022347120805,5940262123258949328
1,organic,1446488424961943503,5940419290323804278
2,referral,145038088022989772,5940420766409453257
3,organic,1450293631537114559,5940402531744356186
4,ad,1450262460605071255,5940275038225886929


To construct paths to purchase, we need to find out which traffic source brought clients to our site each time before they finally bought something. To do this, we need to join the `visits_all` table with itself using `ClientID`.

In [7]:
q = '''
    SELECT 
        ClientID,
        VisitID,
        PurchaseTime,
        DateTime,
        LastTraficSource
    FROM
        (SELECT 
            ClientID,
            DateTime as PurchaseTime, 
            VisitID
        FROM visits_all
        WHERE (ClientID != 0) AND (length(PurchaseID) != 0))
        ALL LEFT JOIN
        (SELECT 
            ClientID,
            LastTraficSource,
            DateTime
        FROM visits_all
        ORDER BY DateTime)
        USING ClientID
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,VisitID,PurchaseTime,DateTime,LastTraficSource
0,1446494115200148125,5935388931880519536,2015-12-02 18:19:23,2015-12-02 18:19:23,direct
1,1446494115200148125,5935388931880519536,2015-12-02 18:19:23,2015-12-31 21:24:48,direct
2,1446494115200148125,5935388931880519536,2015-12-02 18:19:23,2015-12-30 15:56:48,direct
3,1446494115200148125,5935388931880519536,2015-12-02 18:19:23,2015-12-29 20:06:03,direct
4,1446494115200148125,5935388931880519536,2015-12-02 18:19:23,2015-12-28 23:31:21,direct


Now let's group rows by `VisitID` and using function `groupArray` make arrays from traffic sources.

In [8]:
q = '''
    SELECT 
        VisitID,
        groupArray(LastTraficSource) as Sources
    FROM
        (SELECT 
            ClientID,
            DateTime as PurchaseTime, 
            VisitID
        FROM visits_all
        WHERE (ClientID != 0) AND (length(PurchaseID) != 0))
        ALL LEFT JOIN
        (SELECT 
            ClientID,
            LastTraficSource,
            DateTime
        FROM visits_all
        ORDER BY DateTime)
        USING ClientID
    WHERE PurchaseTime >= DateTime
    GROUP BY VisitID
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,VisitID,Sources
0,5942036913160910687,"['ad','ad','ad','ad','ad']"
1,5944887285864662889,"['ad','internal','ad']"
2,5936327790758690505,['ad']
3,5935674958795425569,['social']
4,5937596701532043135,"['organic','organic','referral','ad']"


And finally we need to calculate how many times we see each path.

In [9]:
q = '''
    SELECT 
        Sources, 
        sum(NumPurchases) as TotalNumPurchases
    FROM
        (SELECT 
            any(NumPurchases) as NumPurchases,
            VisitID,
            groupArray(LastTraficSource) as Sources
        FROM
            (SELECT 
                length(PurchaseID) as NumPurchases,
                ClientID,
                DateTime as PurchaseTime, 
                VisitID
            FROM visits_all
            WHERE (ClientID != 0) AND (length(PurchaseID) != 0))
            ALL LEFT JOIN
            (SELECT 
                ClientID,
                LastTraficSource,
                DateTime
            FROM visits_all
            ORDER BY DateTime)
            USING ClientID
        WHERE PurchaseTime >= DateTime
        GROUP BY VisitID)
    WHERE length(Sources) > 1
    GROUP BY Sources 
    ORDER BY TotalNumPurchases DESC
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.42 s


,Sources,TotalNumPurchases
0,"['ad','ad']",2200
1,"['organic','organic']",2159
2,"['organic','ad']",1701
3,"['direct','direct']",989
4,"['ad','ad','ad']",903


We've got our first results :) We have constructed conversion paths for standard Yandex.Metrica traffic sources.

Exciting! But what we actually want is to break the sources into smaller groups – in addition to the default sources breakdown in Yandex.Metrica:
* __CPA__ (`UTMMedium` equals `CPA` or `cpa`)
* __CPC__ (`UTMMedium` equals `cpc`)
* __MarketPlace__ (`UTMMedium` equals `marketplace` or it's ad traffic from Yandex.Market)
* __Retargeting__ (using `DirectConditionType` or by `ref=ga_ref` in URL)
* __Media__ - other ad visits
* Let's group Direct, Saved pages and Internal sources in one group __Direct__


Let's try to do it in ClickHouse.

In [10]:
q = '''
    SELECT 
        LastTraficSource,
        UTMMedium,
        StartURL,
        LastAdvEngine, 
        LastSearchEngine,
        if(
            lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
            lower(UTMMedium),
            if(
                LastTraficSource = 'ad', 
                if(
                    LastAdvEngine = 'market', 
                    'marketplace',
                    if(
                        (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                        'retargeting',
                        'media_ad')
                ),
                if(
                   LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                   'direct',
                   LastTraficSource
                ) 
            )
        ) as Source,
        if(Source = 'organic', 
            if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
            if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                if(
                    LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                    concat(Source, concat(' ', LastAdvEngine)),
                    concat(Source, ' other')
                ),
                Source
            )) as SourceDetailed
    FROM visits_all
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''
get_clickhouse_df(q)

,LastTraficSource,UTMMedium,StartURL,LastAdvEngine,LastSearchEngine,Source,SourceDetailed
0,ad,cpc,http://220-volt.ru/catalog-156860/?ref=yamar&y...,market,yandex,cpc,cpc market
1,organic,NaN,http://m.220-volt.ru/catalog/svetodiodnyj-zana...,NaN,yandex,organic,organic yandex
2,ad,cpc,http://220-volt.ru/catalog-210053/?ref=yamar&y...,market,yandex,cpc,cpc market
3,direct,NaN,http://ulyanovsk.220-volt.ru/catalog-59739/,NaN,NaN,direct,direct
4,direct,NaN,http://m.220-volt.ru/catalog/2-55/,NaN,NaN,direct,direct
5,organic,NaN,http://omsk.220-volt.ru/catalog/,NaN,yandex,organic,organic yandex
6,ad,NaN,http://m.220-volt.ru/catalog-53213/?ref=ga_ret,google_adwords,NaN,media_ad,media_ad google_adwords
7,ad,NaN,http://m.220-volt.ru/catalog-220325/?ref=ga_ret,google_adwords,NaN,media_ad,media_ad google_adwords
8,organic,NaN,http://kemerovo.220-volt.ru/catalog/svarochie-...,NaN,yandex,organic,organic yandex
9,organic,NaN,http://220-volt.ru/catalog-136605/,NaN,google,organic,organic google


I have added to the script for conversion paths new traffic sources algorithm and got stats by new channels.

In [11]:
q = '''
SELECT 
    SourcesDetailed, 
    sum(VisitPurchases) as Purchases
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            VisitID,
            any(ClientID) as ClientID,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases
        FROM
            (SELECT
                ClientID,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                Source,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    ClientID,
                    DateTime as PurchaseTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all 
                WHERE ClientID != 0 AND VisitPurchases != 0)
            ALL LEFT JOIN
                (SELECT
                    ClientID,
                    DateTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed
                 FROM visits_all 
                 ORDER BY DateTime)
            USING ClientID
            WHERE PurchaseTime >= DateTime)
        GROUP BY VisitID)
    GROUP BY SourcesDetailed
    ORDER BY Purchases DESC
    LIMIT 10
    FORMAT TabSeparatedWithNames
'''

%time get_clickhouse_df(q)

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 3.46 s


,SourcesDetailed,Purchases
0,['organic yandex'],8532
1,['direct'],4707
2,['organic google'],3989
3,['cpc market'],3690
4,['cpa other'],2249
5,['cpc yandex'],1966
6,['cpc other'],1531
7,"['organic yandex','organic yandex']",1321
8,"['direct','direct']",1164
9,['email'],1044


## Attribution models

### Last click model

In [12]:
q = '''
    SELECT 
        count() as Visits,
        sum(arraySum(PurchaseRevenue)) as Revenue,
        sum(length(PurchaseID)) as Purchases,
        if(
            lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
            lower(UTMMedium),
            if(
                LastTraficSource = 'ad', 
                if(
                    LastAdvEngine = 'market', 
                    'marketplace',
                    if(
                        (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                        'retargeting',
                        'media_ad')
                ),
                if(
                   LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                   'direct',
                   LastTraficSource
                ) 
            )
        ) as Source,
        if(Source = 'organic', 
            if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
            if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                if(
                    LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                    concat(Source, concat(' ', LastAdvEngine)),
                    concat(Source, ' other')
                ),
                Source
            )) as SourceDetailed
    FROM visits_all
    WHERE ClientID != 0
    GROUP BY Source, SourceDetailed
    FORMAT TabSeparatedWithNames
'''

%time last_raw_df = get_clickhouse_df(q).groupby(['Source', 'SourceDetailed']).sum()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 1.33 s


In [13]:
last_raw_df

Visits     Revenue  Purchases
Source      SourceDetailed                                         
cpa         cpa google_adwords          8144    19420163        179
            cpa market                   299     1636920          9
            cpa other                 324588   533080271       5445
            cpa yandex                  4565     6916381         66
cpc         cpc google_adwords          2152    13963992         81
            cpc market                343667   987963079       7947
            cpc other                 190393   400087849       3279
            cpc yandex                670374   812646786       5528
direct      direct                    882305  2005777498      15712
email       email                     130686   252044411       2526
marketplace marketplace                 5033    13124252        192
media_ad    media_ad google_adwords   360733   301655890       2685
            media_ad other              5880     4600901         60
            media_ad yandex            71485    40893931        461
organic     organic google            601434   569959890       5657
            organic other              56712    97028874        728
            organic yandex           1214740  1466552889      13453
referral    referral                   68278   118097982        938
retargeting retargeting yandex           431      870687          5
social      social                     73483    34991340        334

In [14]:
last_raw_df_norm = last_raw_df.apply(lambda x: 100.*x/last_raw_df.sum(), axis = 1)

In [15]:
last_raw_df_norm

Visits    Revenue  Purchases
Source      SourceDetailed                                          
cpa         cpa google_adwords        0.162380   0.252823   0.274182
            cpa market                0.005962   0.021310   0.013786
            cpa other                 6.471850   6.939962   8.340354
            cpa yandex                0.091020   0.090042   0.101095
cpc         cpc google_adwords        0.042908   0.181792   0.124071
            cpc market                6.852260  12.861902  12.172781
            cpc other                 3.796181   5.208586   5.022593
            cpc yandex               13.366360  10.579528   8.467489
direct      direct                   17.591980  26.112427  24.066784
email       email                     2.605704   3.281267   3.869189
marketplace marketplace               0.100351   0.170859   0.294095
media_ad    media_ad google_adwords   7.192533   3.927139   4.112736
            media_ad other            0.117239   0.059897   0.091905
            media_ad yandex           1.425315   0.532382   0.706135
organic     organic google           11.991788   7.420083   8.665084
            organic other             1.130761   1.263181   1.115111
            organic yandex           24.220289  19.092474  20.606571
referral    referral                  1.361372   1.537471   1.436777
retargeting retargeting yandex        0.008594   0.011335   0.007659
social      social                    1.465153   0.455538   0.511603

### Custom model by behavior pattern

Let's write a basic function to calculate a custom attribution model:
* let's take into account only those sources that brought non-bounce visits
* traffic sources with the goal "Adding product to basket" completed will have a +50% weight
* traffic sources that brought users who browsed a product page will have a +10% weight

In [16]:
def get_attribution_data_sample(i, N, where_condition = ''):
    q = '''
    SELECT 
        sum(VisitRevenue*SourceCoefNorm) as Revenue,
        sum(VisitPurchases*SourceCoefNorm) as Purchases,
        sum(SourceCoefNorm) as Visits,
        Source,
        SourceDetailed
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            groupArray(HasCart) as HasCarts,
            groupArray(HasImpression) as HasImpressions,
            VisitID,
            any(ClientID) as ClientID,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases,
            arrayMap(x, y -> 1 + 0.5*x + 0.1*y, HasCarts, HasImpressions) as SourcesCoefs,
            arraySum(SourcesCoefs) as SourcesCoefsSum,
            arrayMap(x -> x/SourcesCoefsSum, SourcesCoefs) as SourcesCoefsNorm
        FROM
            (SELECT
                ClientID,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                SourceTime,
                Source,
                HasCart,
                HasImpression,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    ClientID,
                    DateTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
                WHERE ClientID != 0)
            ALL LEFT JOIN
                (SELECT
                    ClientID,
                    DateTime as SourceTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed,
                    has(GoalsID, 552829) as HasCart,
                    length(ImpressionsProductID) > 0 as HasImpression
                 FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N} {where_condition}
                 )
            USING ClientID
            WHERE SourceTime <= DateTime
            ORDER BY SourceTime)
        GROUP BY VisitID)
    ARRAY JOIN
        SourcesCoefsNorm as SourceCoefNorm,
        Sources as Source,
        SourcesDetailed as SourceDetailed
    GROUP BY Source, SourceDetailed
    ORDER BY Visits DESC
    FORMAT TabSeparatedWithNames
    '''.format(
        i = i, N = N,
        where_condition = where_condition
    )
    # print q
    return get_clickhouse_df(q)

def get_attribution_data(where_condition=''):
    tmp_dfs = []
    for i in range(10):
        tmp_dfs.append(get_attribution_data_sample(i, 10, where_condition))
    
    return pd.concat(tmp_dfs).groupby(['Source', 'SourceDetailed']).sum()

In [17]:
%time not_bounce_df = get_attribution_data(where_condition = 'WHERE IsBounce = 0')

CPU times: user 44 ms, sys: 8 ms, total: 52 ms
Wall time: 31.2 s


In [18]:
not_bounce_df_norm = not_bounce_df.apply(lambda x: 100.*x/not_bounce_df.sum(), axis = 1)

### Custom time decay model with different weights for each channel

In [19]:
traffic_sources_coeffs = {
    'direct': 0.1,
    'referral': 0.2,
    'social': 0.3,
    'cpa': 0.5,
    'cpc': 0.5,
    'retargeting': 0.3,
    'organic': 0.4,
    'email': 0.4,
    'media_ad': 0.5,
    'marketplace': 0.5
}

In [20]:
def get_attribution_data_pos_sample(i, N, traffic_sources_coeffs, where_condition = ''):
    q = '''
    SELECT 
        sum(VisitRevenue*SourceCoefNorm) as Revenue,
        sum(VisitPurchases*SourceCoefNorm) as Purchases,
        sum(SourceCoefNorm) as Visits,
        Source,
        SourceDetailed
    FROM
        (SELECT 
            groupArray(Source) as Sources,
            groupArray(SourceDetailed) as SourcesDetailed,
            VisitID,
            any(ClientID) as ClientID,
            any(DateTime) as StartTime,
            any(VisitRevenue) as VisitRevenue,
            any(VisitPurchases) as VisitPurchases,
            arrayMap(x -> transform(x, [{traffic_sources}], [{traffic_sources_coeffs}], 0), Sources) as SourcesRawCoefs,
            arrayMap(x -> 1/(length(SourcesRawCoefs) - x), range(length(SourcesRawCoefs))) as PositionCoefs,
            arrayMap(x, y -> x * y, SourcesRawCoefs, PositionCoefs) as SourcesCoefs,
            arraySum(SourcesCoefs) as SourcesCoefsSum,
            arrayMap(x -> x/SourcesCoefsSum, SourcesCoefs) as SourcesCoefsNorm
        FROM
            (SELECT
                ClientID,
                DateTime,
                VisitRevenue,
                VisitPurchases,
                SourceTime,
                Source,
                SourceDetailed,
                VisitID,
                LastSearchEngine,
                LastAdvEngine
            FROM
                (SELECT
                    ClientID,
                    DateTime,
                    arraySum(PurchaseRevenue) as VisitRevenue,
                    length(PurchaseID) as VisitPurchases,
                    LastTraficSource,
                    VisitID,
                    LastSearchEngine,
                    LastAdvEngine
                FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N}
                WHERE ClientID != 0)
            ALL LEFT JOIN
                (SELECT
                    ClientID,
                    DateTime as SourceTime,
                    if(
                        lower(UTMMedium) IN ('cpa', 'cpc', 'marketplace'), 
                        lower(UTMMedium),
                        if(
                            LastTraficSource = 'ad', 
                            if(
                                LastAdvEngine = 'market', 
                                'marketplace',
                                if(
                                    (DirectConditionType = 2) OR (extractURLParameter(StartURL, 'ref') = 'ga_ref'),
                                    'retargeting',
                                    'media_ad')
                            ),
                            if(
                               LastTraficSource IN ('direct', 'saved', 'undefined', 'internal'),
                               'direct',
                               LastTraficSource
                            ) 
                        )
                    ) as Source,
                    if(Source = 'organic', 
                        if(LastSearchEngine IN ('yandex', 'google'), concat('organic ', LastSearchEngine), 'organic other'),
                        if(Source IN ('media_ad', 'retargeting', 'cpa', 'cpc'),
                            if(
                                LastAdvEngine IN ('market', 'google_adwords', 'yandex'), 
                                concat(Source, concat(' ', LastAdvEngine)),
                                concat(Source, ' other')
                            ),
                            Source
                        )) as SourceDetailed
                 FROM visits_all SAMPLE 1/{N} OFFSET {i}/{N} {where_condition})
            USING ClientID
            WHERE SourceTime <= DateTime
            ORDER BY SourceTime)
        GROUP BY VisitID)
    ARRAY JOIN
        SourcesCoefsNorm as SourceCoefNorm,
        Sources as Source,
        SourcesDetailed as SourceDetailed
    GROUP BY Source, SourceDetailed
    ORDER BY Visits DESC
    FORMAT TabSeparatedWithNames
    '''.format(
        traffic_sources = ', '.join(map(lambda x: "'%s'" % x, traffic_sources_coeffs.keys())),
        traffic_sources_coeffs = ', '.join(map(str, traffic_sources_coeffs.values())),
        i = i, N = N,
        where_condition = where_condition
    )
    # print q
    return get_clickhouse_df(q)

def get_attribution_data_pos(traffic_sources_coeffs, where_condition=''):
    tmp_dfs = []
    for i in range(10):
        tmp_dfs.append(get_attribution_data_pos_sample(i, 10, traffic_sources_coeffs, where_condition))
    
    return pd.concat(tmp_dfs).groupby(['Source', 'SourceDetailed']).sum()

In [21]:
%time not_bounce_pos_df = get_attribution_data_pos(traffic_sources_coeffs, where_condition = 'WHERE IsBounce = 0')

CPU times: user 56 ms, sys: 4 ms, total: 60 ms
Wall time: 27.3 s


In [22]:
not_bounce_pos_df_norm = not_bounce_pos_df.apply(lambda x: 100.*x/not_bounce_pos_df.sum(), axis = 1)

## Comparison

For visualization, we'll use the `plot.ly` library that allows to create interactive graphs using just python.

In [31]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)

def show_bar_plot(cmp_df):
    data = []
    for column in cmp_df.columns:
        trace = go.Bar(
            x = cmp_df.index.values,
            y = cmp_df[column].values,
            name = column
        )
        data.append(trace)
    layout = go.Layout({'xaxis': {'tickangle': 45}})
    fig = go.Figure(data = data, layout = layout)
    iplot(fig, show_link=False)

1.12.9


In [24]:
dct = {
    'last click': last_raw_df_norm,
    'behaviour': not_bounce_df_norm,
    'time decay & weights': not_bounce_pos_df_norm,
}

In [25]:
def get_comparison(parameter, detalization, show_delta = False):
    cmp_df = pd.DataFrame()
    for item in dct:
        cmp_df[item] = dct[item].reset_index().groupby(detalization)[parameter].sum()
    cmp_df = cmp_df.sort_values('last click', ascending = False)
    if not show_delta:
        return cmp_df
    cmp_df_norm = cmp_df.apply(lambda x: x - cmp_df['last click']).drop('last click', axis = 1).sort_values('behaviour')
    return cmp_df_norm

In [26]:
show_bar_plot(get_comparison('Revenue', 'Source', show_delta=False))

In [27]:
show_bar_plot(get_comparison('Revenue', 'Source', show_delta=True))

In [28]:
show_bar_plot(get_comparison('Revenue', 'SourceDetailed'))

In [29]:
show_bar_plot(get_comparison('Revenue', 'SourceDetailed', show_delta=True))